In [ ]:
import xarray as xr
from datetime import datetime, timedelta
import glob
import numpy as np
import pandas as pd
import os

# 1. divide

In [ ]:
fnames = sorted(glob.glob('era5_data/0_dloaded/t2m_*.nc'))
output = []
for fname in fnames:
    month = int(fname.split('_')[-2])
    day = int(fname.split('_')[-1].replace('.nc',''))
    mdate = datetime(2020, month, day)
    
    ds = xr.open_dataset(fname, chunks='auto').drop('realization') - 273.15 ## K to C
    ds['hour'] = [np.datetime64(mdate + timedelta(hours=int(c))) for c in ds['hour'].values]
    output.append(ds.rename({'hour':'time'}))
temp_data = xr.concat(output, dim='time')

fnames = sorted(glob.glob('era5_data/0_dloaded/srad_*.nc'))
output = []
for fname in fnames:
    month = int(fname.split('_')[-2])
    day = int(fname.split('_')[-1].replace('.nc',''))
    mdate = datetime(2020, month, day)
    
    ds = xr.open_dataset(fname, chunks='auto').drop('realization')/3600  ## J/m2 to W/m2
    ds['hour'] = [np.datetime64(mdate + timedelta(hours=int(c))) for c in ds['hour'].values]
    output.append(ds.rename({'hour':'time'}))
ssrd_data = xr.concat(output, dim='time')

In [ ]:
for mlat in sorted(ssrd_data['lat'].values):
    
    print(mlat, datetime.now())
    ssrd_tmp = ssrd_data.sel(lat=slice(mlat,mlat))
    temp_tmp = temp_data.sel(lat=slice(mlat,mlat))
    
    ssrd_fname = 'tmp_data/era5_ssrd_' + str(mlat) + '.nc'
    t2m_fname = 'tmp_data/era5_t2m_' + str(mlat) + '.nc'
    
    if not os.path.isfile(ssrd_fname):
        ssrd_tmp.to_netcdf(ssrd_fname, encoding={'ssrd': dict(zlib=True, complevel=5)})
    
    if not os.path.isfile(t2m_fname):
        temp_tmp.to_netcdf(t2m_fname, encoding={'t2m': dict(zlib=True, complevel=5)})
    

# 2. diffuse fraction

In [ ]:
fnames = sorted(glob.glob('era5_data/0_dloaded/fdir_*.nc'))
output = []
for fname in fnames:
    month = int(fname.split('_')[-2])
    day = int(fname.split('_')[-1].replace('.nc',''))
    mdate = datetime(2020, month, day)
    
    ds = xr.open_dataset(fname, chunks='auto').drop('realization')  ## J/m2
    ds['hour'] = [np.datetime64(mdate + timedelta(hours=int(c))) for c in ds['hour'].values]
    output.append(ds.rename({'hour':'time'}))
fdir_data = xr.concat(output, dim='time')

fnames = sorted(glob.glob('era5_data/0_dloaded/srad_*.nc'))
output = []
for fname in fnames:
    month = int(fname.split('_')[-2])
    day = int(fname.split('_')[-1].replace('.nc',''))
    mdate = datetime(2020, month, day)
    
    ds = xr.open_dataset(fname, chunks='auto').drop('realization')  ## J/m2
    ds['hour'] = [np.datetime64(mdate + timedelta(hours=int(c))) for c in ds['hour'].values]
    output.append(ds.rename({'hour':'time'}))
ssrd_data = xr.concat(output, dim='time')

In [ ]:
%time
dfrac_data = 1 - fdir_data['fdir']/ssrd_data['ssrd']
dfrac_data = dfrac_data.clip(0,1).fillna(0).rename('dfrac')
dfrac_data

In [ ]:
for mlat in sorted(dfrac_data['lat'].values):
    
    print(mlat, datetime.now())
    dfrac_tmp = dfrac_data.sel(lat=slice(mlat,mlat))
    
    dfrac_fname = 'tmp_data/era5_dfrac_' + str(mlat) + '.nc'
    
    if not os.path.isfile(dfrac_fname):
        dfrac_tmp.to_netcdf(dfrac_fname, encoding={'dfrac': dict(zlib=True, complevel=5)})